## [Monte Carlo Method](https://harderchoices.com/2018/04/04/monte-carlo-method-in-python/)

Monte Carlo method: 

- reinforcement learnikng method for estimating value
- broad class of computational algorithms that rely on repeated random sampling to obtain numerical results
- using "randomness" to solve our problem

In this notebook, we will use "FrozenLake" game.

__On-policy first-visit MC control__:

- On-policy algorithm: is improving the policy it's working on (opposed to off-policy)[https://stats.stackexchange.com/questions/184657/what-is-the-difference-between-off-policy-and-on-policy-learning]

- first-visit algorithm averages returns for visits following the first vists to given state (opposed to every-visit)

- MC control or Monte carlo conteol. This type of algorithm first estimates values for states, then builds greedy (or near greedy) policy on them just to use the new policy to create another estimate. It loops forever.

In [1]:
import gym
import numpy as np
import operator
from IPython.display import clear_output
from time import sleep
from gym.spaces.tuple_space import Tuple
from gym.envs.registration import register
import random
import itertools
import tqdm

In [2]:
def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env) # 1. 
        
    Q = create_state_action_dictionary(env, policy) # 2.
    returns = {} # 3.
    
    for _ in range(episodes): # 4.
        G = 0 # 5.
        episode = run_game(env=env, policy=policy, display=False) # 6.
        for i in reversed(range(0, len(episode))): # 7.
            s_t, a_t, r_t = episode[i] # 8. 
            state_action = (s_t, a_t)
            G += r_t # 9.
            
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: # 10.
                if returns.get(state_action): # 11.
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # 12.
                
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # 13.
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                max_Q = random.choice(indices)
                
                A_star = max_Q # 14.
                
                for a in policy[s_t].items(): # 15.
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))

    return policy

In [3]:
tqdm.monitor_interval = 0

register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=200
)

register(
    id='FrozenLakeNotSlippery8x8-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '8x8', 'is_slippery': False},
    max_episode_steps=200
)

fl_slippery = {
    'small': 'FrozenLake-v0',
    'big': 'FrozenLake8x8-v0'
}

fl_not_slippery = {
    'small': 'FrozenLakeNotSlippery-v0',
    'big': 'FrozenLakeNotSlippery8x8-v0'
}

In [4]:
def create_environment(slippery=False, big=False):
    if slippery:
        env = gym.make(fl_slippery['big'] if big else fl_slippery['small'])
    else:
        env = gym.make(fl_not_slippery['big'] if big else fl_not_slippery['small'])
    env.reset()
    return env

def create_random_policy(env):
    policy = {}
    for key in range(0, env.observation_space.n):
        current_end = 0
        p = {}
        for action in range(0, env.action_space.n):
            p[action] = 1 / env.action_space.n
        policy[key] = p
    return policy


def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
        Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q    

def run_game(env, policy, display=True):
    env.reset()
    episode = []
    finished = False
    
    while not finished:
        s = env.env.s
            
        if display:
            clear_output(True)
            env.render()
            sleep(0.1)

        timestep = []
        timestep.append(s)
        
        n = random.uniform(0, sum(policy[s].values()))
        top_range = 0
        for prob in policy[s].items():
            top_range += prob[1]
            if n < top_range:
                action = prob[0]
                break   
        
        state, reward, finished, info =  env.step(action)
        timestep.append(action)
        timestep.append(reward)
        
        episode.append(timestep)
        
    if display:
        clear_output(True)
        env.render()
        sleep(0.05)
    
    return episode

def test_policy(policy, env):
    wins = 0
    r = 100
    for i in range(r):
        w = run_game(env, policy, display=False)[-1][-1]
        if w == 1:
            wins += 1
    return wins / r

In [5]:
env = create_environment(slippery=True, big=False)
_ = run_game(env, create_random_policy(env))

  (Right)
SFFF
FHFH
FFFH
HFFG


4x4 not slippery

In [6]:
env = create_environment(slippery=False, big=False)
policy = monte_carlo_e_soft(env, episodes=200)
test_policy(policy, env)

0.94

In [7]:
_ = run_game(env, policy)

  (Right)
SFFF
FHFH
FFFH
HFFG


8x8 not slippery

In [10]:
env = create_environment(slippery=False, big=True)
policy = monte_carlo_e_soft(env, episodes=10000)
test_policy(policy, env)

0.95

In [11]:
_ = run_game(env, policy)

  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


Improvements:

In [12]:
def policy_iterator(env, n, t, epsilon=0.01):
    random_policy = create_random_policy(env)
    random_policy_score = test_policy(random_policy, env)
    best_policy = (random_policy, random_policy_score)
    
    for i in tqdm.tqdm(range(t)):
        new_policy =  monte_carlo_e_soft(env, policy=best_policy[0], episodes=n, epsilon=epsilon)
        new_policy_score = test_policy(new_policy, env)
        if new_policy_score > best_policy[1]:
            best_policy = (new_policy, new_policy_score)
            
    return best_policy

In [14]:
env = create_environment(slippery=True, big=False)
policy, score = policy_iterator(env, 50, 10000, epsilon=0.01)
score

100%|██████████| 10000/10000 [04:49<00:00, 34.49it/s]


0.66